# ML as Function Approximation
* **Function** : An Intro
* **Neural Network** : A function approximator
* **Learning example** : How to Add?

## Introduction to functions

What is a function?
- maps input to output
- can be thought as a curve or a table


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import torch

In [ ]:
%matplotlib inline 

In [ ]:
#Function as a table
x = np.arange(0,4*np.pi,0.1)   # start,stop,step
y = np.sin(x)
list_xy = list(zip(x, y)) 
df_xy = pd.DataFrame(list_xy, columns = ['x', 'y=sin(x)'])  
df_xy.head()

In [ ]:
#Function as a curve
plt.plot(x,y)
plt.xlabel('x')
plt.ylabel('y=sin(x)')
plt.title('sine curve')
plt.show()

### Linear regression as a function

In [ ]:
#Generate data for linear regression
beta=10
n=50
std_dev=100

x = np.random.rand(n) * 100
e = np.random.randn(n) * std_dev
y = x * beta + e
#list_xy = list(zip(x, y)) 
#df_xy = pd.DataFrame(list_xy, columns = ['x', 'y'])  
#df_xy.head()

# plot data
plt.xlabel('x')
plt.ylabel('y')
plt.title('radomly generated 2d points')
plt.scatter(x, y)

In [ ]:
#use linear algebra to find best fit line
#convert data into matrix and vector
X_mat=np.vstack((np.ones(len(x)), x)).T
Y=y

#calculate coefficients for the best fit line
beta_hat = np.linalg.inv(X_mat.T.dot(X_mat)).dot(X_mat.T).dot(Y)
yhat = X_mat.dot(beta_hat)
                 
# plot data and predictions
plt.xlabel('x')
plt.ylabel('y')
plt.title('Best fit line for sample 2d points')
plt.scatter(x, y)
plt.plot(x, yhat, color='red')

## Introduction to Neural Network (Universal Function Approximators)

Given data with labels(i.e. x and y values), we would like to approximate a function f(x) which is very close to y

|Scenario|Data=(x,y)|Function=f(x)|
|---|---|---|
| Best fit line| 2D points  | line approximating 2D points  |
| Image classification | pixels representing picture  | class of image e.g. dog, cat   |
| Audio2Text  | wave representing utterances  | words  |
| Translation  | sentences in multiple languages  | Translator function  |

We are looking for a machine that can generate functions given dataset representing x and y.
[Universal Approximation Theorem](https://medium.com/analytics-vidhya/you-dont-understand-neural-networks-until-you-understand-the-universal-approximation-theorem-85b3e7677126) gives us re-assurance that such a machine exists in our midst.
>_It states that a neural network with one hidden layer containing a sufficient but finite number of neurons can approximate any continuous function to a reasonable accuracy, under certain conditions for activation functions (namely, that they must be sigmoid-like)._

**Windows**

**Artificial Neuron**
<img src="images\BioNeuron.png" />

<br />
<br />

**Artificial Neural Network**
<img src="images\NeuralNetwork.png" />

**Linux**
<br />

**Artificial Neuron**
<img src="images/BioNeuron.png" />

<br />
<br />

**Artificial Neural Network**
<img src="images/NeuralNetwork.png" />

### Terminology 
* Data : (x,y) x is input vector, y is label or output vector. e.g. 2D points.
* Batch : Subset of data that is processed in one pass through the network. 
* $\hat{y}$ : Approximating function that gets us close to y given x. Also referred as f(x) or f(w,x) where w is to be learnt. e.g red line in linear regression.
* Weights : Values of parameters learnt by the network represented by w. e.g. coefficients of best fit line.
* Loss : A function defined over the batch data that measures how far are we from the labels, a measure of _y-$\hat{y}$_. So this is function over approximating function f(x). e.g. summation of sqaure loss over all 2D points.
* Backprop : Process of adjusting weights by using the gradient of loss function to come up with better weights
* Optimizer : Process that assists us to quickly find optimal values of weight so as to minimize loss. This relies on back-propagation.
* Criterion : A human readable measure of model quality. Criterion:Human = Loss: NeuralNetwork

### Deveopment Loops
> We run development loop to come up with the desired approximating function given data. There are 2 loops, inner and outer
> * **Inner**:
> Take a step closer towards desired function by processing a subset of data. This is one pass through the neural network.
>* **Outer**
> Take a step closer towards desired function by processing all available data. We say that 1 epoch has elapsed.


### Inner Loop

***
$\mathbf{\text{Learning objective}}$<br/>
***
\begin{align}
w^{*} = \underset{w}{\textrm{arg min}} \; L(y,\hat{y}) \\
w^{*} = \underset{w}{\textrm{arg min}} \; L(y,f(w,x))
\end{align}

***
$\mathbf{\text{Back Prop}}$<br/>
***
1.&emsp;Initialize model with random weights $$w = w_{0}$$
2. For m = 1 to epochs:<br>
(a)&emsp;Run forward pass by estimating output given current weights $$\hat{y} = f(x,w)$$ 
(b)&emsp;Calculate loss based on the difference between estimated and real outputs. b is batch size. Square loss is an example of simple loss function. $$ L = \sum \limits_{j=1}^b (y-\hat{y})^2 $$ 
(c)&emsp;Calculate loss gradient with respect to weight and use it to update the weight parameter. Alpha is step size. $$ w = w - \nabla_{w} L * \alpha $$
(d)&emsp;Stop when Loss is very small or when patience runs out



### Outer Loop
* Split data into 3 parts : train, validate, test
* For each epoch, build model(aka approximate function) using training data. Then use validate to see the quality of model. Rinse, Repeat.
* Finally, run prediction against test data to see how good the model generalizes

<br/>

## Addition as a mono-lingual translation task 
> Learning how to add using translation function <br/>
> 2 + 3 =5

* data = (2+3, 5)
* function = add(x1,x2)


In [ ]:
def get_random_2numbers(a=100000, b=999999):
    """ sample integers from a uniform distribution """
    
    left = random.randint(a,b)
    right = random.randint(a,b)
                           
    return (left, right)

In [ ]:
def get_sample_algebraic_sequence(left, right, op='+' ):
    if op=='+':
        answer = left + right
        lhs = "{0}+{1}".format(left, right)
        rhs =  "{0}".format(answer)
    elif op=='-':
        answer = left + right
        lhs = "{0}-{1}".format(answer, right)
        rhs =  "{0}".format(left)
    
    return (lhs, rhs)

In [ ]:
def get_sample_algebraic_list(num):
    data_list=[]
    for i in range(num):
        x,y = get_random_2numbers()
        lhs, rhs = get_sample_algebraic_sequence(x, y) 
        data_list.append([lhs,rhs])
    return data_list

In [ ]:
#generate data
data_list = get_sample_algebraic_list(100000)
data_list[:5]

In [ ]:
#load data in a dataframe
df = pd.DataFrame(data_list, columns=['x','y'])
df.head()

In [ ]:
def text_addspace(df, columns_in, save_copy=True):
    """ Adds space between each character """

    for column_in in columns_in:     
        text_processed = df[column_in].apply(lambda s : " ".join(s))

        column_processed = column_in      
        #If column is not to be replaced, save the original column with underscore
        if(save_copy==True):
            column_in_copy = '_' + column_in
            df = df.assign(**{column_in_copy: df[column_in]})

        df = df.assign(**{column_processed: text_processed})

    return df

In [ ]:
#Add space and treat each number as a word
df1= text_addspace(df, ['x','y'])
df1.head()

In [ ]:
#Split dataset into train, validate and test
train, validate, test = np.split(df1.sample(frac=1), [int(.7*len(df1)), int(.9*len(df1))])
train.describe()

In [ ]:
! dir

In [ ]:
train.head()

In [ ]:
#Identify data folder to save input data
work_dir=os.getcwd()
print("Current working directory is: {0}".format(work_dir))
data_dirpath = os.path.join(work_dir, 'data')
print("Data directory is: {0}".format(data_dirpath))

In [ ]:
#Create file paths to save data in 3 sets (train, test, validate). text contains input sequence, summary is output sequence
train_summary_filepath = os.path.join(data_dirpath, 'train.summary')
train_text_filepath    = os.path.join(data_dirpath, 'train.text')
valid_summary_filepath = os.path.join(data_dirpath, 'valid.summary')
valid_text_filepath    = os.path.join(data_dirpath, 'valid.text')
test_summary_filepath  = os.path.join(data_dirpath, 'test.summary')
test_text_filepath     = os.path.join(data_dirpath, 'test.text')

In [ ]:
#Save file on disk
train.to_csv(train_summary_filepath, columns =['y'], header=False, index=False)
train.to_csv(train_text_filepath, columns =['x'], header=False, index=False)
validate.to_csv(valid_summary_filepath, columns =['y'], header=False, index=False)
validate.to_csv(valid_text_filepath, columns =['x'], header=False, index=False)
test.to_csv(test_summary_filepath, columns =['y'], header=False, index=False)
test.to_csv(test_text_filepath, columns =['x'], header=False, index=False)     

### Run training using fairseq in a terminal. Pick command for your operating system

In [ ]:
#pre-process
# windows
! fairseq-preprocess --source-lang text --target-lang summary --trainpref data\train --validpref data\valid --testpref data\test --destdir databin --workers 1

In [ ]:
#pre-process
# linux
! fairseq-preprocess --source-lang text --target-lang summary --trainpref data/train --validpref data/valid --testpref data/test --destdir databin --workers 1

In [ ]:
#train
#windows or linux
! fairseq-train databin --lr 0.0001 --lr-scheduler inverse_sqrt --warmup-init-lr 1e-07 --warmup-updates 1000 \
                        --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 --optimizer adam  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
                        --skip-invalid-size-inputs-valid-test --keep-last-epochs 10 --max-epoch 15 --arch lstm \
                        --save-dir checkpoint

In [ ]:
#generate
#windows
! fairseq-generate databin --path checkpoint\checkpoint_best.pt --gen-subset test --skip-invalid-size-inputs-valid-test \
                 --batch-size 128 --beam 5 --min-len 1 --nbest 1

In [ ]:
#generate
#linux
! fairseq-generate databin --path checkpoint/checkpoint_best.pt --gen-subset test --skip-invalid-size-inputs-valid-test \
                 --batch-size 128 --beam 5 --min-len 1 --nbest 1

In [ ]:
#interactive
#windows
! fairseq-interactive databin --path checkpoint\checkpoint_best.pt --beam 5 --min-len 1 --nbest 1

In [ ]:
#interactive
#linux
! fairseq-interactive databin --path checkpoint/checkpoint_best.pt --beam 5 --min-len 1 --nbest 1